<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# `BaikalETK`: NRPy+-Based BSSN Solver for the Einstein Toolkit

## Author: Zach Etienne

#### Special thanks to Roland Haas for help in answering many implementation questions

## This module generates `BaikalETK`, an [Einstein Toolkit](https://einsteintoolkit.org) thorn for solving Einstein's equations in the BSSN formalism, in Cartesian coordinates. It features SIMD intrinsics and OpenMP support.

**Notebook Status:** <font color='orange'><b> Validated against the Einstein Toolkit `McLachlan` BSSN thorn, both in the context of black hole binary simulations (excellent gravitational waveform agreement) as well as binary neutron star simulations (when parameter `add_stress_energy_source_terms` below is set to `True`). Once plots demonstrating this agreement are added to this tutorial notebook, the validation status will be set to</b></font> <font color='green'><b>Validated</b></font>.

**Validation Notes:** This tutorial notebook has been validated against a trusted Einstein Toolkit thorn, but plots demonstrating its validation have yet to be included in this notebook.

## Introduction

```
How often did my soul cry out:
Come back to Baikal once again?
I still do not know this lake:
To see does not mean to know.
```
[Igor Severyanin](https://en.wikipedia.org/wiki/Igor_Severyanin), [[1]](https://1baikal.ru/en/istoriya/let’s-turn-to-baikal-a-poetic-view).

[Lake Baikal](https://en.wikipedia.org/wiki/Lake_Baikal) is home to the [nerpa seal](https://en.wikipedia.org/wiki/Baikal_seal), NRPy+'s mascot.

This thorn is meant to reproduce the functionality of the `McLachlan` thorn, generated by the [Mathematica](https://www.wolfram.com/mathematica/)-based [Kranc](http://kranccode.org/) code, but using the NRPy+ infrastructure.

### Associated NRPy+ Source Code & Tutorial Modules for this module: 
* [BSSN/ADM_Numerical_Spherical_or_Cartesian_to_BSSNCurvilinear.py](../edit/BSSN/ADM_Numerical_Spherical_or_Cartesian_to_BSSNCurvilinear.py); [\[**tutorial**\]](Tutorial-ADM_Initial_Data-Converting_Exact_ADM_Spherical_or_Cartesian_to_BSSNCurvilinear.ipynb): Spherical/Cartesian ADM$\to$Curvilinear BSSN converter function, for which ADM quantities are assumed given at each gridpoint (i.e., exact, closed-form expressions are not given). This is used to generate BaikalETK's ADM$\to$BSSN function, as in the ETK spacetime evolution modules are to assume that initial data are given as ADM quantities in the Cartesian basis at each gridpoint.
* [BSSN/ADM_in_terms_of_BSSN.py](../edit/BSSN/ADM_in_terms_of_BSSN.py); [\[**tutorial**\]](Tutorial-ADM_in_terms_of_BSSN.ipynb): Constructs ADM quantities in terms of BSSN quantities (in arbitrary curvilinear coordinates, though we use Cartesian here). This is used to generate BaikalETK's BSSN$\to$ADM function, which make ADM variables available to diagnostic thorns within the ETK.
* [BSSN/BSSN_constraints.py](../edit/BSSN/BSSN_constraints.py); [\[**tutorial**\]](Tutorial-BSSN_constraints.ipynb): Hamiltonian constraint in BSSN curvilinear basis/coordinates
* [BSSN/BSSN_RHSs.py](../edit/BSSN/BSSN_RHSs.py); [\[**tutorial**\]](Tutorial-BSSN_time_evolution-BSSN_RHSs.ipynb): Generates the right-hand sides for the BSSN evolution equations in singular, curvilinear coordinates
* [BSSN/BSSN_gauge_RHSs.py](../edit/BSSN/BSSN_gauge_RHSs.py); [\[**tutorial**\]](Tutorial-BSSN_time_evolution-BSSN_gauge_RHSs.ipynb): Generates the right-hand sides for the BSSN gauge evolution equations in singular, curvilinear coordinates

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Initialize needed Python/NRPy+ modules
1. [Step 2](#bssn): NRPy+-generated C code kernels for BSSN spacetime solve
    1. [Step 2.a](#bssnrhs): BSSN RHS expressions
    1. [Step 2.b](#hammomconstraints): Hamiltonian & momentum constraints
    1. [Step 2.c](#gamconstraint): Enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint (in Cartesian coordinates, $\det{\hat{\gamma}_{ij}}=1$)
    1. [Step 2.d](#parallel_codegen): Generate all the above C code kernels in parallel
1. [Step 3](#cclfiles): CCL files - Define how this module interacts and interfaces with the wider Einstein Toolkit infrastructure
    1. [Step 3.a](#paramccl): `param.ccl`: specify free parameters within `BaikalETK`
    1. [Step 3.b](#interfaceccl): `interface.ccl`: define needed gridfunctions; provide keywords denoting what this thorn provides and what it should inherit from other thorns
    1. [Step 3.c](#scheduleccl): `schedule.ccl`:schedule all functions used within `BaikalETK`, specify data dependencies within said functions, and allocate memory for gridfunctions
1. [Step 4](#cdrivers): C driver functions for ETK registration & NRPy+-generated kernels
    1. [Step 4.a](#etkfunctions): Needed ETK functions: Banner, Symmetry registration, Parameter sanity check,  Method of Lines (`MoL`) registration, Boundary condition
    1. [Step 4.b](#bssnadmconversions): BSSN $\leftrightarrow$ ADM conversions
        1. [Step 4.b.i](#admtobssn): ADM $\to$ BSSN
        1. [Step 4.b.ii](#bssntoadm): BSSN $\to$ ADM
    1. [Step 4.c](#bssnrhss) Evaluate BSSN right-hand-sides (RHSs)
        1. [Step 4.c.i](#ricci): Evaluate Ricci tensor
        1. [Step 4.c.ii](#bssnrhssricciinput): Evaluate BSSN RHSs, using Ricci tensor as input 
    1. [Step 4.d](#enforcegammahatconstraint): Enforcing conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint (in Cartesian coordinates, $\det{\hat{\gamma}_{ij}}=1$)
    1. [Step 4.e](#diagnostics): Diagnostics: Computing Hamiltonian & momentum constraints
    1. [Step 4.f](#t4uu): `driver_BSSN_T4UU()`: Compute $T^{\mu\nu}$ from `TmunuBase`'s $T_{\mu\nu}$
    1. [Step 4.g](#makecodedefn): `make.code.defn`: List of all C driver functions needed to compile `BaikalETK`
1. [Step 5](#code_validation): Code Validation
    1. [Step 5.a](#self_validation): Validation against [BaikalETK.BaikalETK_Pymodule](../edit/BaikalETK/BaikalETK_Pymodule.py) module
1. [Step 6](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

In [ ]:
# step -1, find nrpy
import sys
sys.path.append("nrpytutorial")

In [ ]:
# Step 0 define my own looping constructs
def loop_int(kernel):
    return """Loop::loop_int<0,0,0>(cctkGH, [&](const Loop::PointDesc &p){
    const int i0 = p.i;
    const int i1 = p.j;
    const int i2 = p.k;
"""+kernel+"""
});\n"""
def loop_all(kernel):
    return """Loop::loop_all<0,0,0>(cctkGH, [&](const Loop::PointDesc &p){
    const int i0 = p.i;
    const int i1 = p.j;
    const int i2 = p.k;
"""+kernel+"""
});\n"""

<a id='initializenrpy'></a>

# Step 1: Initialize needed Python/NRPy+ modules \[Back to [top](#toc)\]

$$\label{initializenrpy}$$

In [ ]:
# Step 1: Import needed core NRPy+ modules
from outputC import *            # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions

# Create directory for BaikalETK thorn & subdirectories in case they don't exist.
outrootdir = "../"
cmd.mkdir(os.path.join(outrootdir))
outdir = os.path.join(outrootdir,"src") # Main C code output directory
cmd.mkdir(outdir)

# Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Enable stress-energy terms?
add_stress_energy_source_terms = False

# Default Kreiss-Oliger dissipation strength
default_KO_strength = 0.1

# Step 2: Set some core parameters, including CoordSystem MoL timestepping algorithm,
#                                 FD order, floating point precision, and CFL factor:
# Choices are: Spherical, SinhSpherical, SinhSphericalv2, Cylindrical, SinhCylindrical, 
#              SymTP, SinhSymTP
# NOTE: Only CoordSystem == Cartesian makes sense here; new 
#       boundary conditions are needed within the ETK for 
#       Spherical, etc. coordinates.
CoordSystem     = "Cartesian"

par.set_parval_from_str("reference_metric::CoordSystem",CoordSystem)
rfm.reference_metric() # Create ReU, ReDD needed for rescaling B-L initial data, generating BSSN RHSs, etc.

# Set the standard 1+log lapse condition
LapseCondition  = "OnePlusLog"
# Set the standard, second-order advecting-shift, Gamma-driving shift condition
ShiftCondition  = "GammaDriving2ndOrder_NoCovariant"

FD_order  = 4           # Finite difference order: even numbers only, starting with 2. 12 is generally unstable
REAL      = "CCTK_REAL" # Set REAL to CCTK_REAL, the ETK data type for 
                        # floating point precision (typically `double`)
# Set finite differencing order:
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", FD_order)

# Set the gridfunction memory access type to ETK-like, so that finite_difference
#    knows how to read and write gridfunctions from/to memory.
par.set_parval_from_str("grid::GridFuncMemAccess","CarpetX")

<a id='bssn'></a>

# Step 2: Output C code for BSSN spacetime solve \[Back to [top](#toc)\]
$$\label{bssn}$$

<a id='bssnrhs'></a>

## Step 2.a: BSSN RHS expressions \[Back to [top](#toc)\]
$$\label{bssnrhs}$$

`BaikalETK` implements a fully covariant version of the BSSN 3+1 decomposition of Einstein's equations of general relativity, which is fully documented within NRPy+ ([start here](Tutorial-BSSN_formulation.ipynb)). However, especially if you are a newcomer to the field of numerical relativity, you may also find the following lectures and papers useful for understanding the adopted formalism:

* Mathematical foundations of BSSN and 3+1 initial value problem decompositions of Einstein's equations:
    * [Thomas Baumgarte's lectures on mathematical formulation of numerical relativity](https://www.youtube.com/watch?v=t3uo2R-yu4o&list=PLRVOWML3TL_djTd_nsTlq5aJjJET42Qke)
    * [Yuichiro Sekiguchi's introduction to BSSN](http://www2.yukawa.kyoto-u.ac.jp/~yuichiro.sekiguchi/3+1.pdf) 
* Extensions to the standard BSSN approach used in NRPy+
    * [Brown's covariant "Lagrangian" formalism of BSSN](https://arxiv.org/abs/0902.3652)
    * [BSSN in spherical coordinates, using the reference-metric approach of Baumgarte, Montero, Cordero-Carrión, and Müller (2012)](https://arxiv.org/abs/1211.6632)
    * [BSSN in generic curvilinear coordinates, using the extended reference-metric approach of Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658)
    
Here, we simply call the [BSSN.BSSN_RHSs](../edit/BSSN/BSSN_RHSs.py); [\[**tutorial**\]](Tutorial-BSSN_time_evolution-BSSN_RHSs.ipynb) and [BSSN.BSSN_gauge_RHSs](../edit/BSSN/BSSN_gauge_RHSs.py); [\[**tutorial**\]](Tutorial-BSSN_time_evolution-BSSN_gauge_RHSs.ipynb) NRPy+ Python modules to generate the symbolic expressions, add Kreiss-Oliger dissipation, and then output the finite-difference C code form of the equations using NRPy+'s [finite_difference](../edit/finite_difference.py) ([**tutorial**](Tutorial-Finite_Difference_Derivatives.ipynb)) C code generation module.

In [ ]:
import time # Standard Python module; useful for benchmarking below expression & code generation.

import BSSN.BSSN_RHSs as rhs
import BSSN.BSSN_gauge_RHSs as gaugerhs
par.set_parval_from_str("BSSN.BSSN_gauge_RHSs::ShiftEvolutionOption", ShiftCondition)
par.set_parval_from_str("BSSN.BSSN_gauge_RHSs::LapseEvolutionOption", LapseCondition)

print("Generating symbolic expressions for BSSN RHSs...")
start = time.time()
# Enable rfm_precompute infrastructure, which results in 
#   BSSN RHSs that are free of transcendental functions,
#   even in curvilinear coordinates, so long as 
#   ConformalFactor is set to "W" (default).
cmd.mkdir(os.path.join(outdir,"rfm_files/"))
par.set_parval_from_str("reference_metric::enable_rfm_precompute","True")
par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir",os.path.join(outdir,"rfm_files/"))

# Evaluate BSSN + BSSN gauge RHSs with rfm_precompute enabled:
import BSSN.BSSN_quantities as Bq
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic","True")

rhs.BSSN_RHSs()

if add_stress_energy_source_terms == True:
    T4UU = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","T4UU","sym01",DIM=4)
    import BSSN.BSSN_stress_energy_source_terms as Bsest
    Bsest.BSSN_source_terms_for_BSSN_RHSs(T4UU)
    rhs.trK_rhs += Bsest.sourceterm_trK_rhs
    for i in range(DIM):
        # Needed for Gamma-driving shift RHSs:
        rhs.Lambdabar_rhsU[i] += Bsest.sourceterm_Lambdabar_rhsU[i]
        # Needed for BSSN RHSs:
        rhs.lambda_rhsU[i]    += Bsest.sourceterm_lambda_rhsU[i]
        for j in range(DIM):
            rhs.a_rhsDD[i][j] += Bsest.sourceterm_a_rhsDD[i][j]

gaugerhs.BSSN_gauge_RHSs()

# Add Kreiss-Oliger dissipation to the BSSN RHSs:
thismodule = "KO_Dissipation"
diss_strength = par.Cparameters("REAL", thismodule, "diss_strength", default_KO_strength)

alpha_dKOD = ixp.declarerank1("alpha_dKOD")
cf_dKOD    = ixp.declarerank1("cf_dKOD")
trK_dKOD   = ixp.declarerank1("trK_dKOD")
betU_dKOD    = ixp.declarerank2("betU_dKOD","nosym")
vetU_dKOD    = ixp.declarerank2("vetU_dKOD","nosym")
lambdaU_dKOD = ixp.declarerank2("lambdaU_dKOD","nosym")
aDD_dKOD = ixp.declarerank3("aDD_dKOD","sym01")
hDD_dKOD = ixp.declarerank3("hDD_dKOD","sym01")
for k in range(DIM):
    gaugerhs.alpha_rhs += diss_strength*alpha_dKOD[k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
    rhs.cf_rhs         += diss_strength*   cf_dKOD[k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
    rhs.trK_rhs        += diss_strength*  trK_dKOD[k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
    for i in range(DIM):
        if "2ndOrder" in ShiftCondition:
            gaugerhs.bet_rhsU[i] += diss_strength*   betU_dKOD[i][k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
        gaugerhs.vet_rhsU[i]     += diss_strength*   vetU_dKOD[i][k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
        rhs.lambda_rhsU[i]       += diss_strength*lambdaU_dKOD[i][k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
        for j in range(DIM):
            rhs.a_rhsDD[i][j] += diss_strength*aDD_dKOD[i][j][k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
            rhs.h_rhsDD[i][j] += diss_strength*hDD_dKOD[i][j][k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]

# We use betaU as our upwinding control vector:
Bq.BSSN_basic_tensors()
betaU = Bq.betaU

import BSSN.Enforce_Detgammabar_Constraint as EGC
enforce_detg_constraint_symb_expressions = EGC.Enforce_Detgammabar_Constraint_symb_expressions()

# Next compute Ricci tensor
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic","False")
Bq.RicciBar__gammabarDD_dHatD__DGammaUDD__DGammaU()

# Now that we are finished with all the rfm hatted
#           quantities in generic precomputed functional
#           form, let's restore them to their closed-
#           form expressions.
par.set_parval_from_str("reference_metric::enable_rfm_precompute","False") # Reset to False to disable rfm_precompute.
rfm.ref_metric__hatted_quantities()
end = time.time()
print("Finished BSSN symbolic expressions in "+str(end-start)+" seconds.")


def BSSN_RHSs():
    print("Generating C code for BSSN RHSs in "+par.parval_from_str("reference_metric::CoordSystem")+" coordinates.")
    start = time.time()

    BSSN_evol_rhss = [ \
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD00"),rhs=rhs.a_rhsDD[0][0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD01"),rhs=rhs.a_rhsDD[0][1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD02"),rhs=rhs.a_rhsDD[0][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD11"),rhs=rhs.a_rhsDD[1][1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD12"),rhs=rhs.a_rhsDD[1][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD22"),rhs=rhs.a_rhsDD[2][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","alpha"),rhs=gaugerhs.alpha_rhs),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","betU0"),rhs=gaugerhs.bet_rhsU[0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","betU1"),rhs=gaugerhs.bet_rhsU[1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","betU2"),rhs=gaugerhs.bet_rhsU[2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","cf"),   rhs=rhs.cf_rhs),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD00"),rhs=rhs.h_rhsDD[0][0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD01"),rhs=rhs.h_rhsDD[0][1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD02"),rhs=rhs.h_rhsDD[0][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD11"),rhs=rhs.h_rhsDD[1][1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD12"),rhs=rhs.h_rhsDD[1][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD22"),rhs=rhs.h_rhsDD[2][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","lambdaU0"),rhs=rhs.lambda_rhsU[0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","lambdaU1"),rhs=rhs.lambda_rhsU[1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","lambdaU2"),rhs=rhs.lambda_rhsU[2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","trK"),  rhs=rhs.trK_rhs),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","vetU0"),rhs=gaugerhs.vet_rhsU[0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","vetU1"),rhs=gaugerhs.vet_rhsU[1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","vetU2"),rhs=gaugerhs.vet_rhsU[2]) ]

    BSSN_RHSs_string = fin.FD_outputC("returnstring",BSSN_evol_rhss, params="outCverbose=False,SIMD_enable=False",
                                      upwindcontrolvec=betaU)

    with open(os.path.join(outdir,"BSSN_RHSs.h"), "w") as file:
        file.write(loop_int(BSSN_RHSs_string))
    end = time.time()
    print("Finished BSSN_RHS C codegen in " + str(end - start) + " seconds.")

def Ricci():
    print("Generating C code for Ricci tensor in "+par.parval_from_str("reference_metric::CoordSystem")+" coordinates.")
    start = time.time()
    Ricci_string = fin.FD_outputC("returnstring",
                                  [lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD00"),rhs=Bq.RbarDD[0][0]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD01"),rhs=Bq.RbarDD[0][1]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD02"),rhs=Bq.RbarDD[0][2]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD11"),rhs=Bq.RbarDD[1][1]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD12"),rhs=Bq.RbarDD[1][2]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD22"),rhs=Bq.RbarDD[2][2])],
                                   params="outCverbose=False,SIMD_enable=False")
    with open(os.path.join(outdir,"BSSN_Ricci.h"), "w") as file:
        file.write(loop_int(Ricci_string))
    end = time.time()
    print("Finished Ricci C codegen in " + str(end - start) + " seconds.")

<a id='hammomconstraints'></a>

## Step 2.b: Hamiltonian & momentum constraints \[Back to [top](#toc)\]
$$\label{hammomconstraints}$$

Next output the C code for evaluating the Hamiltonian & momentum constraints [(**Tutorial**)](Tutorial-BSSN_constraints.ipynb). In the absence of numerical error, this constraint should evaluate to zero. However it does not due to numerical (typically truncation and roundoff) error. Therefore it is useful to measure the Hamiltonian & momentum constraint violation to gauge the accuracy of our simulation, and, ultimately determine whether errors are dominated by numerical finite differencing (truncation) error as expected.

In [ ]:
# First register the Hamiltonian as a gridfunction.
H  = gri.register_gridfunctions("AUX","H")
MU = ixp.register_gridfunctions_for_single_rank1("AUX", "MU")

# Then define the Hamiltonian constraint and output the optimized C code.
import BSSN.BSSN_constraints as bssncon

def BSSNconstraints():
    bssncon.BSSN_constraints(add_T4UUmunu_source_terms=False)
    if add_stress_energy_source_terms == True:
#         T4UU = gri.register_gridfunctions_for_single_rank2("AUXEVOL","T4UU","sym01",DIM=4)
        import BSSN.BSSN_stress_energy_source_terms as Bsest
        Bsest.BSSN_source_terms_for_BSSN_constraints(T4UU)
        bssncon.H += Bsest.sourceterm_H
        for i in range(DIM):
            bssncon.MU[i] += Bsest.sourceterm_MU[i]
        
    start = time.time()
    print("Generating optimized C code for Ham. & mom. constraints. May take a while, depending on CoordSystem.")
    Ham_mom_string = fin.FD_outputC("returnstring", 
                                    [lhrh(lhs=gri.gfaccess("aux_gfs", "H"),   rhs=bssncon.H),
                                     lhrh(lhs=gri.gfaccess("aux_gfs", "MU0"), rhs=bssncon.MU[0]),
                                     lhrh(lhs=gri.gfaccess("aux_gfs", "MU1"), rhs=bssncon.MU[1]),
                                     lhrh(lhs=gri.gfaccess("aux_gfs", "MU2"), rhs=bssncon.MU[2])],
                                    params="outCverbose=False")

    with open(os.path.join(outdir,"BSSN_constraints.h"), "w") as file:
        file.write(loop_int(Ham_mom_string))
    end = time.time()
    print("Finished Hamiltonian & momentum constraint C codegen in " + str(end - start) + " seconds.")

<a id='gamconstraint'></a>

## Step 2.c: Enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint (in Cartesian coordinates, $\det{\hat{\gamma}_{ij}}=1$) \[Back to [top](#toc)\]
$$\label{gamconstraint}$$

Then enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint (Eq. 53 of [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658)), as [documented in the corresponding NRPy+ tutorial notebook](Tutorial-BSSN-Enforcing_Determinant_gammabar_equals_gammahat_Constraint.ipynb)

Applying curvilinear boundary conditions should affect the initial data at the outer boundary, and will in general cause the $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint to be violated there. Thus after we apply these boundary conditions, we must always call the routine for enforcing the $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint:

In [ ]:
def gammadet():
    start = time.time()
    print("Generating optimized C code for gamma constraint. May take a while, depending on CoordSystem.")
    enforce_gammadet_string = fin.FD_outputC("returnstring", enforce_detg_constraint_symb_expressions,
                                             params="outCverbose=False,preindent=0,includebraces=False")

    with open(os.path.join(outdir,"enforce_detgammabar_constraint.h"), "w") as file:
        file.write(loop_all(enforce_gammadet_string))
    end = time.time()
    print("Finished gamma constraint C codegen in " + str(end - start) + " seconds.")

<a id='parallel_codegen'></a>

## Step 2.d: Generate all C codes in parallel \[Back to [top](#toc)\]
$$\label{parallel_codegen}$$


In [ ]:
# Step 0: Import the multiprocessing module.
import multiprocessing

# Step 1: Create a list of functions we wish to evaluate in parallel
funcs = [BSSN_RHSs,Ricci,BSSNconstraints,gammadet]
# Step 1.a: Define master function for parallelization.
#           Note that lambdifying this doesn't work in Python 3
def master_func(arg):
    funcs[arg]()

# Step 2: Evaluate list of functions in parallel if allowed; 
#         otherwise fallback to serial evaluation:
try:
    if __name__ == '__main__':
        pool = multiprocessing.Pool()
        pool.map(master_func,range(len(funcs)))
except:
    # If multiprocessing didn't work for whatever reason,
    #        evaluate functions in serial.
    for func in funcs:
        func()

<a id='cclfiles'></a>

# Step 3: ETK `ccl` file generation \[Back to [top](#toc)\]
$$\label{cclfiles}$$

<a id='paramccl'></a>

## Step 3.a: `param.ccl`: specify free parameters within `BaikalETK` \[Back to [top](#toc)\]
$$\label{paramccl}$$

All parameters necessary for the computation of the BSSN right-hand side (RHS) expressions are registered within NRPy+; we use this information to automatically generate `param.ccl`. NRPy+ also specifies default values for each parameter. 

More information on `param.ccl` syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3).

In [ ]:
def keep_param__return_type(paramtuple):
    keep_param = True # We'll not set some parameters in param.ccl; 
                      #   e.g., those that should be #define'd like M_PI.
    typestring = ""
    # Separate thorns within the ETK take care of grid/coordinate parameters;
    #   thus we ignore NRPy+ grid/coordinate parameters:
    if paramtuple.module == "grid" or paramtuple.module == "reference_metric":
        keep_param = False

    partype = paramtuple.type
    if partype == "bool":
        typestring += "BOOLEAN "
    elif partype == "REAL":
        if paramtuple.defaultval != 1e300: # 1e300 is a magic value indicating that the C parameter should be mutable
            typestring += "CCTK_REAL "
        else:
            keep_param = False
    elif partype == "int":
        typestring += "CCTK_INT "
    elif partype == "#define":
        keep_param = False
    elif partype == "char":
        # FIXME: char/string parameter types should in principle be supported
        print("Error: parameter "+paramtuple.module+"::"+paramtuple.parname+
              " has unsupported type: \""+ paramtuple.type + "\"")
        sys.exit(1)
    else:
        print("Error: parameter "+paramtuple.module+"::"+paramtuple.parname+
              " has unsupported type: \""+ paramtuple.type + "\"")
        sys.exit(1)
    return keep_param, typestring

with open(os.path.join(outrootdir,"param.ccl"), "w") as file:
    file.write("""
# This param.ccl file was automatically generated by NRPy+. 
#   You are advised against modifying it directly.

shares: ADMBase

restricted:
""")
    paramccl_str = ""
    for i in range(len(par.glb_Cparams_list)):
        # keep_param is a boolean indicating whether we should accept or reject
        #    the parameter. singleparstring will contain the string indicating
        #    the variable type.
        keep_param, singleparstring = keep_param__return_type(par.glb_Cparams_list[i])

        if keep_param:
            parname = par.glb_Cparams_list[i].parname
            partype = par.glb_Cparams_list[i].type
            singleparstring += parname + " \""+ parname +" (see NRPy+ for parameter definition)\"\n"
            singleparstring += "{\n"
            if partype != "bool":
                singleparstring += " *:* :: \"All values accepted. NRPy+ does not restrict the allowed ranges of parameters yet.\"\n"
            singleparstring += "} "+str(par.glb_Cparams_list[i].defaultval)+"\n\n"
            
            paramccl_str += singleparstring
    file.write(paramccl_str)

<a id='interfaceccl'></a>

## Step 3.b: `interface.ccl`: define needed gridfunctions; provide keywords denoting what this thorn provides and what it should inherit from other thorns \[Back to [top](#toc)\]
$$\label{interfaceccl}$$

`interface.ccl` declares all gridfunctions and determines how `BaikalETK` interacts with other Einstein Toolkit thorns.

The [official Einstein Toolkit (Cactus) documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManual.html) defines what must/should be included in an `interface.ccl` file [**here**](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 

In [ ]:
# First construct lists of the basic gridfunctions used in NRPy+. 
#    Each type will be its own group in BaikalETK.
evol_gfs_list    = []
auxevol_gfs_list = []
aux_gfs_list     = []
for i in range(len(gri.glb_gridfcs_list)):
    if gri.glb_gridfcs_list[i].gftype == "EVOL":
        evol_gfs_list.append(   gri.glb_gridfcs_list[i].name+"GF")

    if gri.glb_gridfcs_list[i].gftype == "AUX":
        aux_gfs_list.append(    gri.glb_gridfcs_list[i].name+"GF")

    if gri.glb_gridfcs_list[i].gftype == "AUXEVOL":
        auxevol_gfs_list.append(gri.glb_gridfcs_list[i].name+"GF")

# NRPy+'s finite-difference code generator assumes gridfunctions
#    are alphabetized; not sorting may result in unnecessary
#    cache misses.
evol_gfs_list.sort()
aux_gfs_list.sort()
auxevol_gfs_list.sort()
        
with open(os.path.join(outrootdir,"interface.ccl"), "w") as file:
    file.write("""
# With "implements", we give our thorn its unique name.
implements: BaikalETK

# By "inheriting" other thorns, we tell the Toolkit that we 
#   will rely on variables/function that exist within those
#   functions. 
inherits: ADMBase ODESolvers\n""")
    if add_stress_energy_source_terms == True:
        file.write("inherits:   TmunuBase")
    file.write("""
public:
""")
    
    # Next we declare gridfunctions based on their corresponding gridfunction groups as registered within NRPy+
    
    def output_list_of_gfs(gfs_list,description="User did not provide description"):
        gfsstr = "    "
        for i in range(len(gfs_list)):
            gfsstr += gfs_list[i]
            if i != len(gfs_list)-1:
                gfsstr += "," # This is a comma-separated list of gridfunctions
            else:
                gfsstr += "\n} \""+description+"\"\n\n"
        return gfsstr
    # First EVOL type:
    file.write("CCTK_REAL evol_variables type = GF Timelevels=3 TAGS=\'rhs=\"evol_variables_rhs\"'\n{\n")
    file.write(output_list_of_gfs(evol_gfs_list,"BSSN evolved gridfunctions"))
    # Second EVOL right-hand-sides
    file.write("CCTK_REAL evol_variables_rhs type = GF Timelevels=1 TAGS=\'InterpNumTimelevels=1 prolongation=\"none\"\'\n{\n")
    rhs_list = []
    for gf in evol_gfs_list:
        rhs_list.append(gf.replace("GF","")+"_rhsGF")
    file.write(output_list_of_gfs(rhs_list,"right-hand-side storage for BSSN evolved gridfunctions"))
    # Then AUX type:
    file.write("CCTK_REAL aux_variables type = GF Timelevels=3\n{\n")
    file.write(output_list_of_gfs(aux_gfs_list,"Auxiliary gridfunctions for BSSN diagnostics"))
    # Finally, AUXEVOL type:
    file.write("CCTK_REAL auxevol_variables type = GF Timelevels=1 TAGS=\'InterpNumTimelevels=1 prolongation=\"none\"\'\n{\n")
    file.write(output_list_of_gfs(auxevol_gfs_list,"Auxiliary gridfunctions needed for evaluating the BSSN RHSs"))

<a id='scheduleccl'></a>

## Step 3.c: `schedule.ccl`: schedule all functions used within `BaikalETK`, specify data dependencies within said functions, and allocate memory for gridfunctions \[Back to [top](#toc)\]
$$\label{scheduleccl}$$

Official documentation on constructing ETK `schedule.ccl` files is found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4).

In [ ]:
with open(os.path.join(outrootdir,"schedule.ccl"), "w") as file:
    file.write("""
# First allocate storage for all ADMBase gridfunctions, which are needed by NRPy+
STORAGE: ADMBase::metric, ADMBase::curv, ADMBase::lapse, ADMBase::shift

# Next allocate storage for all 3 gridfunction groups used in BaikalETK
STORAGE: evol_variables[3]     # Evolution variables
STORAGE: evol_variables_rhs[1] # Variables storing right-hand-sides
STORAGE: aux_variables[3]      # Diagnostics variables
STORAGE: auxevol_variables[1]  # Single-timelevel storage of variables needed for evolutions.

# The following scheduler is based on Lean/LeanBSSNMoL/schedule.ccl

schedule BaikalETK_Banner at STARTUP
{
  LANG: C
  OPTIONS: meta
} "Output ASCII art banner"

schedule BaikalETK_ADM_to_BSSN at CCTK_INITIAL after ADMBase_PostInitial
{
  LANG: C
  OPTIONS: Local
  SYNC: evol_variables
} "Convert initial data into BSSN variables"

# MoL: compute RHSs, etc
""")
    if add_stress_energy_source_terms == True:
        file.write("""
schedule driver_BSSN_T4UU in ODESolvers_RHS as BaikalETK_T4UU before BaikalETK_BSSN_to_ADM
{
  LANG: C
} "MoL: Compute T4UU, needed for BSSN RHSs."

schedule BaikalETK_BSSN_to_ADM in ODESolvers_RHS after BaikalETK_T4UU before BaikalETK_Ricci
{
  LANG: C
} "Perform BSSN-to-ADM conversion. Needed for HydroBase coupling."
""")
    file.write("""
schedule driver_pt1_BSSN_Ricci in ODESolvers_RHS as BaikalETK_Ricci before BaikalETK_RHS
{
  LANG: C
} "MoL: Compute Ricci tensor"

schedule driver_pt2_BSSN_RHSs in ODESolvers_RHS as BaikalETK_RHS after BaikalETK_Ricci
{
  LANG: C
} "MoL: Evaluate BSSN RHSs"

schedule enforce_detgammabar_constraint in MoL_PostStep before BC_Update
{
  LANG: C
} "Enforce detgammabar = detgammahat (= 1 in Cartesian)"

# Next update ADM quantities

schedule BaikalETK_BSSN_to_ADM in MoL_PostStep after BaikalETK_ApplyBCs before ADMBase_SetADMVars
{
  LANG: C
  OPTIONS: Local
} "Perform BSSN-to-ADM conversion. Useful for diagnostics."

# Compute Hamiltonian & momentum constraints
""")
    if add_stress_energy_source_terms == True:
        file.write("""
schedule group MoL_PseudoEvolution in POSTSTEP
{
} "Compute constraints etc."

schedule driver_BSSN_T4UU in MoL_PseudoEvolution before BaikalETK_BSSN_constraints
{
  LANG: C
  OPTIONS: Local
} "MoL_PseudoEvolution: Compute T4UU, needed for BSSN constraints"
""")
    file.write("""

schedule BaikalETK_BSSN_constraints in MoL_PseudoEvolution
{
  LANG: C
  OPTIONS: Local
} "Compute BSSN (Hamiltonian and momentum) constraints"

""")

<a id='cdrivers'></a>

# Step 4: C driver functions for ETK registration & NRPy+-generated kernels \[Back to [top](#toc)\]
$$\label{cdrivers}$$

Now that we have constructed the basic C code kernels and the needed Einstein Toolkit `ccl` files, we next write the driver functions for registering `BaikalETK` within the Toolkit and the C code kernels. Each of these driver functions is called directly from [`schedule.ccl`](#scheduleccl).

<a id='etkfunctions'></a>
## Step 4.a: Needed ETK functions: Banner, Symmetry registration, Parameter sanity check,  Method of Lines (`MoL`) registration, Boundary condition \[Back to [top](#toc)\]
$$\label{etkfunctions}$$

### To-do: Parameter sanity check function. E.g., error should be thrown if `cctk_nghostzones[]` is set too small for the chosen finite-differencing order within NRPy+.

In [ ]:
make_code_defn_list = []
def append_to_make_code_defn_list(filename):
    if filename not in make_code_defn_list:
        make_code_defn_list.append(filename)
    return os.path.join(outdir,filename)

In [ ]:
# First the ETK banner code, proudly showing the NRPy+ banner
import NRPy_logo as logo

with open(append_to_make_code_defn_list("Banner.cc"),"w") as file:
    file.write("""
#include <stdio.h>

#include "cctk.h"

int BaikalETK_Banner(void) 
{
    """)
    logostr = logo.print_logo(print_to_stdout=False)
    file.write("printf(\"BaikalETK: another Einstein Toolkit thorn generated by\\n\");\n")
    for line in logostr.splitlines():
        file.write("    printf(\""+line+"\\n\");\n")
    file.write("return 0;\n")
    file.write("}\n")

In [ ]:
# Next register symmetries
with open(append_to_make_code_defn_list("zero_rhss.cc"),"w") as file:
    file.write("""
#include <loop.hxx>    

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
#include "Symmetry.h"

void BaikalETK_zero_rhss(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
""")
    rhss_loopers = ""
    for gf in rhs_list:
        rhss_loopers += "Loop::GF3D<CCTK_REAL, 0, 0, 0> "+gf+"_(cctkGH, "+gf+");\n"
    file.write(rhss_loopers)
    
    set_rhss_to_zero = ""
    for gf in rhs_list:
        set_rhss_to_zero += gf+"_(i0, i1, i2) = 0.0;\n"
    
    file.write(loop_all(set_rhss_to_zero))
    file.write("}\n")

<a id='bssnadmconversions'></a>

## Step 4.b: BSSN $\leftrightarrow$ ADM conversions \[Back to [top](#toc)\]
$$\label{bssnadmconversions}$$

<a id='admtobssn'></a>

### Step 4.b.i: ADM $\to$ BSSN \[Back to [top](#toc)\]
$$\label{admtobssn}$$

Initial data in the Einstein Toolkit are given in terms of [ADM quantities](https://en.wikipedia.org/wiki/ADM_formalism), so a conversion is necessary so the quantities are in terms of BSSN variables used for evolving the initial data forward in time.

In [ ]:
# First we convert from ADM to BSSN, as is required to convert initial data 
#    (given using) ADM quantities, to the BSSN evolved variables
import BSSN.ADM_Numerical_Spherical_or_Cartesian_to_BSSNCurvilinear as atob
IDhDD,IDaDD,IDtrK,IDvetU,IDbetU,IDalpha,IDcf,IDlambdaU = \
    atob.Convert_Spherical_or_Cartesian_ADM_to_BSSN_curvilinear("Cartesian","DoNotOutputADMInputFunction",outdir)

alphaSphorCart   = gri.register_gridfunctions(                 "AUXEVOL", "alphaSphorCart")
betaSphorCartU   = ixp.register_gridfunctions_for_single_rank1("AUXEVOL", "betaSphorCartU")
BSphorCartU      = ixp.register_gridfunctions_for_single_rank1("AUXEVOL", "BSphorCartU")
gammaSphorCartDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL", "gammaSphorCartDD", "sym01")
KSphorCartDD     = ixp.register_gridfunctions_for_single_rank2("AUXEVOL", "KSphorCartDD", "sym01")

# Step : Output ADM to BSSN conversion.
with open(append_to_make_code_defn_list("ADM_to_BSSN.cc"), "w") as file:
    file.write("""
#include <math.h>

#include <loop.hxx>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

void BaikalETK_ADM_to_BSSN(CCTK_ARGUMENTS) {
    DECLARE_CCTK_ARGUMENTS;
    DECLARE_CCTK_PARAMETERS;
    
""")
#   """    // make copies of ADMBase variables where we expect them"""    
#    copy_admbase_vars = []
#    copy_admbase_vars.append("alphaSphorCartGF[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)] = alp[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)];\n")
#    # It's ugly if we output code in the following ordering, so we'll first
#    #   output to a string and then sort the string to beautify the code a bit.
#    for i in range(DIM):
#        copy_admbase_vars.append("betaSphorCartU"+str(i)+"GF[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)] = beta"+chr(ord('x')+i)+"[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)];\n")
#        copy_admbase_vars.append("BSphorCartU"+str(i)+"GF[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)] = dtbeta"+chr(ord('x')+i)+"[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)];\n")
#        for j in range(i,DIM):
#            copy_admbase_vars.append("gammaSphorCartDD"+str(i)+str(j)+"GF[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)] = g"+chr(ord('x')+i)+chr(ord('x')+j)+"[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)];\n")
#            copy_admbase_vars.append("KSphorCartDD"+str(i)+str(j)+"GF[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)] = k"+chr(ord('x')+i)+chr(ord('x')+j)+"[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)];\n")
#    copy_admbase_vars.sort()
#    file.write(lp.loop(["i2","i1","i0"],["0", "0", "0"],
#                       ["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],
#                       ["1","1","1"],
#                        ["#pragma omp parallel for","","",],"","".join(copy_admbase_vars)))

    file.write("""
CCTK_REAL *alphaSphorCartGF = alp;
""")
    # It's ugly if we output code in the following ordering, so we'll first
    #   output to a string and then sort the string to beautify the code a bit.
    outstr = []
    for i in range(DIM):
        outstr.append("    CCTK_REAL *betaSphorCartU"+str(i)+"GF = beta"+chr(ord('x')+i)+";\n")
        outstr.append("    CCTK_REAL *BSphorCartU"+str(i)+"GF = dtbeta"+chr(ord('x')+i)+";\n")
        for j in range(i,DIM):
            outstr.append("    CCTK_REAL *gammaSphorCartDD"+str(i)+str(j)+"GF = g"+chr(ord('x')+i)+chr(ord('x')+j)+";\n")
            outstr.append("    CCTK_REAL *KSphorCartDD"+str(i)+str(j)+"GF = k"+chr(ord('x')+i)+chr(ord('x')+j)+";\n")
    outstr.sort()
    for line in outstr:
        file.write(line)

    adm_to_bssn_gfs = []
    adm_to_bssn_gfs.append("alphaSphorCart"+"GF")
    adm_to_bssn_gfs.append("trK"+"GF")
    adm_to_bssn_gfs.append("alpha"+"GF")
    adm_to_bssn_gfs.append("cf"+"GF")
    for i in range(DIM):
        adm_to_bssn_gfs.append("betaSphorCartU"+str(i)+"GF")
        adm_to_bssn_gfs.append("BSphorCartU"+str(i)+"GF")
        adm_to_bssn_gfs.append("vetU"+str(i)+"GF")
        adm_to_bssn_gfs.append("betU"+str(i)+"GF")
        adm_to_bssn_gfs.append("lambdaU"+str(i)+"GF")
        for j in range(i,DIM):
            adm_to_bssn_gfs.append("gammaSphorCartDD"+str(i)+str(j)+"GF")
            adm_to_bssn_gfs.append("KSphorCartDD"+str(i)+str(j)+"GF")
            adm_to_bssn_gfs.append("hDD"+str(i)+str(j)+"GF")
            adm_to_bssn_gfs.append("aDD"+str(i)+str(j)+"GF")
    for gf in adm_to_bssn_gfs:
        file.write("Loop::GF3D<CCTK_REAL,0,0,0> "+gf+"_(cctkGH,"+gf+");\n")
    file.write("\n")

    file.write("""
    const CCTK_REAL invdx0 = 1.0/CCTK_DELTA_SPACE(0);
    const CCTK_REAL invdx1 = 1.0/CCTK_DELTA_SPACE(1);
    const CCTK_REAL invdx2 = 1.0/CCTK_DELTA_SPACE(2);
""")

    all_but_lambdaU_expressions = [
        lhrh(lhs=gri.gfaccess("in_gfs","hDD00"),rhs=IDhDD[0][0]),
        lhrh(lhs=gri.gfaccess("in_gfs","hDD01"),rhs=IDhDD[0][1]),
        lhrh(lhs=gri.gfaccess("in_gfs","hDD02"),rhs=IDhDD[0][2]),
        lhrh(lhs=gri.gfaccess("in_gfs","hDD11"),rhs=IDhDD[1][1]),
        lhrh(lhs=gri.gfaccess("in_gfs","hDD12"),rhs=IDhDD[1][2]),
        lhrh(lhs=gri.gfaccess("in_gfs","hDD22"),rhs=IDhDD[2][2]),
        lhrh(lhs=gri.gfaccess("in_gfs","aDD00"),rhs=IDaDD[0][0]),
        lhrh(lhs=gri.gfaccess("in_gfs","aDD01"),rhs=IDaDD[0][1]),
        lhrh(lhs=gri.gfaccess("in_gfs","aDD02"),rhs=IDaDD[0][2]),
        lhrh(lhs=gri.gfaccess("in_gfs","aDD11"),rhs=IDaDD[1][1]),
        lhrh(lhs=gri.gfaccess("in_gfs","aDD12"),rhs=IDaDD[1][2]),
        lhrh(lhs=gri.gfaccess("in_gfs","aDD22"),rhs=IDaDD[2][2]),
        lhrh(lhs=gri.gfaccess("in_gfs","trK"),rhs=IDtrK),
        lhrh(lhs=gri.gfaccess("in_gfs","vetU0"),rhs=IDvetU[0]),
        lhrh(lhs=gri.gfaccess("in_gfs","vetU1"),rhs=IDvetU[1]),
        lhrh(lhs=gri.gfaccess("in_gfs","vetU2"),rhs=IDvetU[2]),
        lhrh(lhs=gri.gfaccess("in_gfs","betU0"),rhs=IDbetU[0]),
        lhrh(lhs=gri.gfaccess("in_gfs","betU1"),rhs=IDbetU[1]),
        lhrh(lhs=gri.gfaccess("in_gfs","betU2"),rhs=IDbetU[2]),
        lhrh(lhs=gri.gfaccess("in_gfs","alpha"),rhs=IDalpha),
        lhrh(lhs=gri.gfaccess("in_gfs","cf"),rhs=IDcf)]
                                   
    outCparams = "preindent=1,outCfileaccess=a,outCverbose=False,includebraces=False"
    all_but_lambdaU_outC = fin.FD_outputC("returnstring",all_but_lambdaU_expressions, outCparams)
    file.write(loop_all(all_but_lambdaU_outC))

    outCparams = "preindent=1,outCfileaccess=a,outCverbose=False,includebraces=False"
    lambdaU_expressions = [lhrh(lhs=gri.gfaccess("in_gfs","lambdaU0"),rhs=IDlambdaU[0]),
                           lhrh(lhs=gri.gfaccess("in_gfs","lambdaU1"),rhs=IDlambdaU[1]),
                           lhrh(lhs=gri.gfaccess("in_gfs","lambdaU2"),rhs=IDlambdaU[2])]
    lambdaU_expressions_FDout = fin.FD_outputC("returnstring",lambdaU_expressions, outCparams)

    file.write(loop_all(lambdaU_expressions_FDout))
    
    file.write("""
    //ExtrapolateGammas(cctkGH,lambdaU0GF);
    //ExtrapolateGammas(cctkGH,lambdaU1GF);
    //ExtrapolateGammas(cctkGH,lambdaU2GF);
}
""")

<a id='bssntoadm'></a>

### Step 4.b.ii: BSSN $\to$ ADM \[Back to [top](#toc)\]
$$\label{bssntoadm}$$

All modules (thorns) in the Einstein Toolkit that deal with spacetime quantities do so via the core `ADMBase` module, which assumes variables are written in ADM form. Therefore, in order for `BaikalETK` to interface properly with the rest of the Toolkit, its native BSSN variables must be converted to ADM quantities.

In [ ]:
import BSSN.ADM_in_terms_of_BSSN as btoa
btoa.ADM_in_terms_of_BSSN()
Bq.BSSN_basic_tensors() # Gives us betaU & BU

with open(append_to_make_code_defn_list("BSSN_to_ADM.cc"), "w") as file:
    file.write("""
#include <loop.hxx>

#include <math.h>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

void BaikalETK_BSSN_to_ADM(CCTK_ARGUMENTS) {
    DECLARE_CCTK_ARGUMENTS;
    DECLARE_CCTK_PARAMETERS;
    
""")
    bssn_to_adm_gfs = []
    bssn_to_adm_gfs.append("trK"+"GF")
    bssn_to_adm_gfs.append("alpha"+"GF")
    bssn_to_adm_gfs.append("alp")
    bssn_to_adm_gfs.append("cf"+"GF")
    for i in range(DIM):
        bssn_to_adm_gfs.append("beta"+chr(ord('x')+i))
        bssn_to_adm_gfs.append("dtbeta"+chr(ord('x')+i))
        bssn_to_adm_gfs.append("vetU"+str(i)+"GF")
        bssn_to_adm_gfs.append("betU"+str(i)+"GF")
        for j in range(i,DIM):
            bssn_to_adm_gfs.append("g"+chr(ord('x')+i)+chr(ord('x')+j))
            bssn_to_adm_gfs.append("k"+chr(ord('x')+i)+chr(ord('x')+j))
            bssn_to_adm_gfs.append("hDD"+str(i)+str(j)+"GF")
            bssn_to_adm_gfs.append("aDD"+str(i)+str(j)+"GF")
    for gf in bssn_to_adm_gfs:
        file.write("Loop::GF3D<CCTK_REAL,0,0,0> "+gf+"_(cctkGH,"+gf+");\n")
    file.write("\n")    

    btoa_lhrh = []
    for i in range(DIM):
        for j in range(i,DIM):
            btoa_lhrh.append(lhrh(lhs="g"+chr(ord('x')+i)+chr(ord('x')+j)+"_(p.I)",
                                  rhs=btoa.gammaDD[i][j]))
    for i in range(DIM):
        for j in range(i,DIM):
            btoa_lhrh.append(lhrh(lhs="k"+chr(ord('x')+i)+chr(ord('x')+j)+"_(p.I)",
                                  rhs=btoa.KDD[i][j]))
    btoa_lhrh.append(lhrh(lhs="alp_(p.I)",rhs=Bq.alpha))
            
    for i in range(DIM):
        btoa_lhrh.append(lhrh(lhs="beta"+chr(ord('x')+i)+"_(p.I)",
                                  rhs=Bq.betaU[i]))

    for i in range(DIM):
        btoa_lhrh.append(lhrh(lhs="dtbeta"+chr(ord('x')+i)+"_(p.I)",
                                  rhs=Bq.BU[i]))
    
    outCparams = "preindent=1,outCfileaccess=a,outCverbose=False,includebraces=False"
    bssn_to_adm_Ccode = fin.FD_outputC("returnstring",btoa_lhrh, outCparams)
    file.write(loop_all(bssn_to_adm_Ccode))

    file.write("}\n")

<a id='bssnrhss'></a>

## Step 4.c: Evaluate BSSN right-hand-sides (RHSs) \[Back to [top](#toc)\]
$$\label{bssnrhss}$$

<a id='ricci'></a>

### Step 4.c.i: Evaluate Ricci tensor \[Back to [top](#toc)\]
$$\label{ricci}$$

To slightly optimize the performance of `BaikalETK`'s BSSN solver, we split the computation of the [complicated expressions for the Ricci tensor $\\bar{R}_{ij}$](Tutorial-BSSN_quantities.ipynb#rbar) into its own function, and then use the result when evaluating the BSSN right-hand-side (RHS) expressions.

In [ ]:
with open(append_to_make_code_defn_list("driver_pt1_BSSN_Ricci.cc"), "w") as file:
    file.write("""
#include <math.h>

#include <loop.hxx>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

void driver_pt1_BSSN_Ricci(CCTK_ARGUMENTS) {
    DECLARE_CCTK_ARGUMENTS;
    
    const CCTK_REAL invdx0 = 1.0/CCTK_DELTA_SPACE(0);
    const CCTK_REAL invdx1 = 1.0/CCTK_DELTA_SPACE(1);
    const CCTK_REAL invdx2 = 1.0/CCTK_DELTA_SPACE(2);
""")
    pt1_bssn_ricci_gfs = []
    for i in range(DIM):
        pt1_bssn_ricci_gfs.append("lambdaU"+str(i)+"GF")
        for j in range(i,DIM):
            pt1_bssn_ricci_gfs.append("hDD"+str(i)+str(j)+"GF")
            pt1_bssn_ricci_gfs.append("RbarDD"+str(i)+str(j)+"GF")
    for gf in pt1_bssn_ricci_gfs:
        file.write("Loop::GF3D<CCTK_REAL,0,0,0> "+gf+"_(cctkGH,"+gf+");\n")
    file.write("\n")    

    file.write("""#include "BSSN_Ricci.h"
}\n""")

<a id='bssnrhssricciinput'></a>

### Step 4.c.ii: Evaluate BSSN RHSs, using Ricci tensor as input \[Back to [top](#toc)\]
$$\label{bssnrhssricciinput}$$

Next we construct the driver function for evaluating the BSSN RHSs, which make use of the Ricci tensor $\bar{R}_{ij}$, which has just been computed.

In [ ]:
with open(append_to_make_code_defn_list("driver_pt2_BSSN_RHSs.cc"), "w") as file:
    file.write("""
#include <math.h>

#include <loop.hxx>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

//void BSSN_RHSs()

#define UPWIND_ALG(UpwindVecU) UpwindVecU > 0.0 ? 1.0 : 0.0

void driver_pt2_BSSN_RHSs(CCTK_ARGUMENTS) {
    DECLARE_CCTK_ARGUMENTS;
    DECLARE_CCTK_PARAMETERS;

    const CCTK_REAL invdx0 = 1.0/CCTK_DELTA_SPACE(0);
    const CCTK_REAL invdx1 = 1.0/CCTK_DELTA_SPACE(1);
    const CCTK_REAL invdx2 = 1.0/CCTK_DELTA_SPACE(2);
""")
    pt2_bssn_rhs_gfs = []
    pt2_bssn_rhs_gfs.append("trK"+"GF")
    pt2_bssn_rhs_gfs.append("trK"+"_rhsGF")
    pt2_bssn_rhs_gfs.append("alpha"+"GF")
    pt2_bssn_rhs_gfs.append("alpha"+"_rhsGF")
    pt2_bssn_rhs_gfs.append("cf"+"GF")
    pt2_bssn_rhs_gfs.append("cf"+"_rhsGF")
    for i in range(DIM):
        pt2_bssn_rhs_gfs.append("lambdaU"+str(i)+"GF")
        pt2_bssn_rhs_gfs.append("lambdaU"+str(i)+"_rhsGF")
        pt2_bssn_rhs_gfs.append("vetU"+str(i)+"GF")
        pt2_bssn_rhs_gfs.append("vetU"+str(i)+"_rhsGF")
        pt2_bssn_rhs_gfs.append("betU"+str(i)+"GF")
        pt2_bssn_rhs_gfs.append("betU"+str(i)+"_rhsGF")
        for j in range(i,DIM):
            pt2_bssn_rhs_gfs.append("hDD"+str(i)+str(j)+"GF")
            pt2_bssn_rhs_gfs.append("aDD"+str(i)+str(j)+"GF")
            pt2_bssn_rhs_gfs.append("hDD"+str(i)+str(j)+"_rhsGF")
            pt2_bssn_rhs_gfs.append("aDD"+str(i)+str(j)+"_rhsGF")
            pt2_bssn_rhs_gfs.append("RbarDD"+str(i)+str(j)+"GF")
    for gf in pt2_bssn_rhs_gfs:
        file.write("Loop::GF3D<CCTK_REAL,0,0,0> "+gf+"_(cctkGH,"+gf+");\n")
    file.write("\n")    

    file.write("""
#include "BSSN_RHSs.h"
}\n""")

<a id='enforcegammahatconstraint'></a>

## Step 4.d: Enforcing conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint (in Cartesian coordinates, $\det{\hat{\gamma}_{ij}}=1$) \[Back to [top](#toc)\]
$$\label{enforcegammahatconstraint}$$

Here we construct the driver function for enforcing the conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint. The BSSN equations are not strongly hyperbolic if this condition is not set.

In [ ]:
with open(append_to_make_code_defn_list("enforce_detgammabar_constraint.cc"), "w") as file:
    file.write("""
#include <math.h>

#include <loop.hxx>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

void enforce_detgammabar_constraint(CCTK_ARGUMENTS) {
    DECLARE_CCTK_ARGUMENTS;
    DECLARE_CCTK_PARAMETERS;
""")
    enforce_detgammabar_constraint_gfs = []
    for i in range(DIM):
        for j in range(i,DIM):
            enforce_detgammabar_constraint_gfs.append("hDD"+str(i)+str(j)+"GF")
    for gf in enforce_detgammabar_constraint_gfs:
        file.write("Loop::GF3D<CCTK_REAL,0,0,0> "+gf+"_(cctkGH,"+gf+");\n")
    file.write("\n")    

    file.write("""
#include "enforce_detgammabar_constraint.h"
}\n""")

<a id='diagnostics'></a>

## Step 4.e: Diagnostics: Computing Hamiltonian & momentum constraints \[Back to [top](#toc)\]
$$\label{diagnostics}$$

The BSSN Hamiltonian & momentum constraints are useful diagnostics of a numerical-relativity calculation's health, as both should converge to zero with increasing numerical resolution. Here we construct the driver function.

In [ ]:
with open(append_to_make_code_defn_list("BSSN_constraints.cc"), "w") as file:
    file.write("""
#include <math.h>

#include <loop.hxx>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

void BaikalETK_BSSN_constraints(CCTK_ARGUMENTS) {
    DECLARE_CCTK_ARGUMENTS;
    DECLARE_CCTK_PARAMETERS;

    const CCTK_REAL invdx0 = 1.0/CCTK_DELTA_SPACE(0);
    const CCTK_REAL invdx1 = 1.0/CCTK_DELTA_SPACE(1);
    const CCTK_REAL invdx2 = 1.0/CCTK_DELTA_SPACE(2);
""")
    BSSN_constraints_gfs = []
    BSSN_constraints_gfs.append("trK"+"GF")
    BSSN_constraints_gfs.append("cf"+"GF")
    BSSN_constraints_gfs.append("H"+"GF")
    for i in range(DIM):
        BSSN_constraints_gfs.append("lambdaU"+str(i)+"GF")
        BSSN_constraints_gfs.append("MU"+str(i)+"GF")
        for j in range(i,DIM):
            BSSN_constraints_gfs.append("hDD"+str(i)+str(j)+"GF")
            BSSN_constraints_gfs.append("aDD"+str(i)+str(j)+"GF")
    for gf in BSSN_constraints_gfs:
        file.write("Loop::GF3D<CCTK_REAL,0,0,0> "+gf+"_(cctkGH,"+gf+");\n")
    file.write("\n")    

    file.write("""
#include "BSSN_constraints.h"
}\n""")

<a id='t4uu'></a>

## Step 4.f: `driver_BSSN_T4UU()`: Compute $T^{\mu\nu}$ from `TmunuBase`'s $T_{\mu\nu}$ \[Back to [top](#toc)\]
$$\label{t4uu}$$

Here we implement $T^{\mu\nu} = g^{\mu \delta} g^{\nu \gamma} T_{\delta \gamma}.$

In [ ]:
if add_stress_energy_source_terms == True:
    # Declare T4DD as a set of gridfunctions. These won't 
    #    actually appear in interface.ccl, as interface.ccl 
    #    was set above. Thus before calling the code output
    #    by FD_outputC(), we'll have to set pointers
    #    to the actual gridfunctions they reference.
    #    (In this case the eTab's.)
    T4DD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","T4DD","sym01",DIM=4)
    import BSSN.ADMBSSN_tofrom_4metric as AB4m
    AB4m.g4UU_ito_BSSN_or_ADM("BSSN")

    T4UUraised = ixp.zerorank2(DIM=4)
    for mu in range(4):
        for nu in range(4):
            for delta in range(4):
                for gamma in range(4):
                    T4UUraised[mu][nu] += AB4m.g4UU[mu][delta]*AB4m.g4UU[nu][gamma]*T4DD[delta][gamma]
    
    T4UU_expressions = [
        lhrh(lhs=gri.gfaccess("in_gfs","T4UU00"),rhs=T4UUraised[0][0]),
        lhrh(lhs=gri.gfaccess("in_gfs","T4UU01"),rhs=T4UUraised[0][1]),
        lhrh(lhs=gri.gfaccess("in_gfs","T4UU02"),rhs=T4UUraised[0][2]),
        lhrh(lhs=gri.gfaccess("in_gfs","T4UU03"),rhs=T4UUraised[0][3]),
        lhrh(lhs=gri.gfaccess("in_gfs","T4UU11"),rhs=T4UUraised[1][1]),
        lhrh(lhs=gri.gfaccess("in_gfs","T4UU12"),rhs=T4UUraised[1][2]),
        lhrh(lhs=gri.gfaccess("in_gfs","T4UU13"),rhs=T4UUraised[1][3]),
        lhrh(lhs=gri.gfaccess("in_gfs","T4UU22"),rhs=T4UUraised[2][2]),
        lhrh(lhs=gri.gfaccess("in_gfs","T4UU23"),rhs=T4UUraised[2][3]),
        lhrh(lhs=gri.gfaccess("in_gfs","T4UU33"),rhs=T4UUraised[3][3])]

    outCparams = "outCverbose=False,includebraces=False,preindent=2,SIMD_enable=False"
    T4UUstr = fin.FD_outputC("returnstring",T4UU_expressions, outCparams)
    T4UUstr_loop = loop_all(T4UUstr)

    with open(append_to_make_code_defn_list("driver_BSSN_T4UU.cc"), "w") as file:
        file.write("""
#include <math.h>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

void driver_BSSN_T4UU(CCTK_ARGUMENTS) {
    DECLARE_CCTK_ARGUMENTS;
    DECLARE_CCTK_PARAMETERS;

    const CCTK_REAL *restrict T4DD00GF = eTtt;
    const CCTK_REAL *restrict T4DD01GF = eTtx;
    const CCTK_REAL *restrict T4DD02GF = eTty;
    const CCTK_REAL *restrict T4DD03GF = eTtz;
    const CCTK_REAL *restrict T4DD11GF = eTxx;
    const CCTK_REAL *restrict T4DD12GF = eTxy;
    const CCTK_REAL *restrict T4DD13GF = eTxz;
    const CCTK_REAL *restrict T4DD22GF = eTyy;
    const CCTK_REAL *restrict T4DD23GF = eTyz;
    const CCTK_REAL *restrict T4DD33GF = eTzz;
"""+T4UUstr_loop+"""
}\n""")

<a id='makecodedefn'></a>

## Step 4.g: `make.code.defn`: List of all C driver functions needed to compile `BaikalETK` \[Back to [top](#toc)\]
$$\label{makecodedefn}$$

When constructing each C code driver function above, we called the `append_to_make_code_defn_list()` function, which built a list of each C code driver file. We'll now add each of those files to the `make.code.defn` file, used by the Einstein Toolkit's build system.

In [ ]:
with open(os.path.join(outdir,"make.code.defn"), "w") as file:
    file.write("""
# Main make.code.defn file for thorn BaikalETK

# Source files in this directory
SRCS =""")
    filestring = ""
    for i in range(len(make_code_defn_list)):
        filestring += "      "+make_code_defn_list[i]
        if i != len(make_code_defn_list)-1:
            filestring += " \\\n"
        else:
            filestring += "\n"
    file.write(filestring)